# Topic modeling with BERTopic

BERTopic est un modèle de sujet qui exploite les techniques de clustering et une variante de TF-IDF basée sur les classes pour la génération des sujets :
1. création des document embeddings à l'aide d'un modèle de langage pré-entraîné 
2. réduction de la dimensionnalité des embeddings des documents avant de créer des clusters de documents sémantiquement similaires, chacun représentant un sujet distinct
3. application d'une version de TF-IDF basée sur les classes afin d'extraire la représentation des sujets (*topic representation*) de chaque sujet.

*Topic* est un cluster de documents sémantiquement similaires
*Topic representation* est la façon dont les sujets sont décrits ou labélisés, ici par un set de mots extraits des documents du cluster.

=> https://medialab.sciencespo.fr/actu/les-enjeux-de-linformation-a-lere-numerique-vus-par-les-francais/

In [4]:
import polars as pl
from nltk.corpus import stopwords
from spacy.lang.fr.stop_words import STOP_WORDS
# from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lang.zh.stop_words import STOP_WORDS
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/lydia/miniconda3/envs/semantique/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/lydia/miniconda3/envs/semantique/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/lydia/miniconda3/envs/semantique/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/lydia/miniconda3/envs/semantique/lib/python3.10/site-packages/traitlets/config/appl

In [ ]:
# nous avons besoin de `protobuf` version 3.20 pour `SentenceTransformer` avec camembert
# !pip install protobuf==3.20.0

# Préparation des données

Les modèles de sujet avec transformers et word-embedding ne demande, en théorie, aucun prétraitement des données. Cependant, il peut s'avérer utile de normaliser les données, d'enlever les balises HTML (quand ce n'est pas déjà fait), etc. Dans le cas de certains corpus, la suppression de mots spécifiques présents dans tout le corpus peut également s'avérer utile. Après cela, on peut transformer notre dataframe en list de documents.

In [5]:
docs = pl.read_csv("corpus.csv").with_columns(
    pl.col("captions").str.normalize("NFC")
).filter(
    pl.col("captions").is_not_null()
).get_column(
    "captions"
).to_list() # remplacer captions par la colonne contenant vos publications
docs[0]

FileNotFoundError: No such file or directory (os error 2): corpus.csv

In [ ]:
len(docs)

Après cela, nous préparons une liste de mots vides, que nous exclurons de la représentation des sujets.

In [ ]:
stoplist = list(STOP_WORDS)
# ADDITIONAL_STOPWORDS = []
# stoplist.extend(ADDITIONAL_STOPWORDS)
stoplist[:5]

# Vectoriser les données

### Word embeddings

La première étape consiste à transformer une chaîne (phrase) en un tableau de nombres (vecteur), autrement dit à vectoriser le texte.
BERTopic peut convertir les documents en embeddings. Cependant, ce processus peut être très coûteux. Il est donc possible de calculer ces embeddings une seule fois et de les transmettre à BERTopic pour éviter de les calculer à chaque fois.

=> https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
=> https://huggingface.co/spaces/mteb/leaderboard 

In [ ]:
sentences = ["ah ouais c'est ça votre argument ultime", "aujourd'hui ils sont contraignants"]

embedding_model = SentenceTransformer("dangvantuan/sentence-camembert-base") # ("all-MiniLM-L6-v2")
example_embeddings = embedding_model.encode(sentences, show_progress_bar=True)

print(f"Sentence 1 has {len(sentences[0])} characters, and the embedding is {len(example_embeddings[0])} long.")
print(f"Sentence 2 has {len(sentences[1])} characters, and the embedding is {len(example_embeddings[1])} long.")

Comme le montre la cellule ci-dessus, les embeddings continues créées par le transformateur de phrases `dangvantuan/sentence-camembert-base` ont la même longueur, malgré des phrases de longueurs différentes. Ces embeddings continues ont la même longueur car, plutôt que de représenter directement les mots sous forme de nombres (c'est-à-dire de « sac de mots »), le transformateur crée une représentation riche prenant en compte 768 dimensions.

=> https://github.com/MaartenGr/BERTopic/discussions/822

### UMAP

BERTopic utilise généralement un algorithme de réduction de dimensionnalité (*dimensionality reduction algorithm*) pour réduire la taille des embeddings. Ceci permet d'éviter, dans une certaine mesure, le fléau de la dimension (https://en.wikipedia.org/wiki/Curse_of_dimensionality). 

UMAP est utilisé pour réduire l'espace dimensionnel. Cependant, il affiche par défaut un comportement stochastique qui produit des résultats différents à chaque exécution. Pour éviter cela pendant la période de test, nous pouvons définir un random_state du modèle avant de le transmettre à BERTopic.

```python
UMAP(angular_rp_forest=True, metric='cosine', n_components=10, n_neighbors=30, min_dist=0.1, random_state=42)
```

=> https://umap-learn.readthedocs.io/en/latest/parameters.html

### HDBSCAN

`nr_topics` est le paramètre qui permet de contrôler le nombre de sujets en fusionnant les sujets après leur création. Il permet de créer un nombre fixe de sujets. Il est toutefois conseillé de contrôler le nombre de sujets via le modèle de cluster, HDBSCAN par défaut. HDBSCAN possède un paramètre, `min_cluster_size`, qui contrôle indirectement le nombre de sujets créés. Un `min_cluster_size` élevé génère moins de sujets, tandis qu'un `min_cluster_size` faible en génère davantage.

```python
HDBSCAN(min_cluster_size=13, min_samples=3, prediction_data=True, metric='euclidean', cluster_selection_method='eom')
```

=> https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html

### CountVectorizer

La représentation par défaut des sujets est calculée via c-TF-IDF. Cependant, c-TF-IDF s'appuie sur CountVectorizer, qui convertit le texte en unités. Grâce à CountVectorizer, nous pouvons effectuer plusieurs opérations : supprimer les mots vides, ignorer les mots rares, augmenter la range des n-gram. En d'autres termes, nous pouvons prétraiter les représentations des sujets après l'attribution des documents aux sujets. Cela n'aura aucune incidence sur le processus de clustering.

La tokenisation se fait automatiquement au niveau des espaces, pour le chinois il faut rajouter :

```python
import jieba

def tokenize_zh(text):
    words = jieba.lcut(text)
    return words

vectorizer = CountVectorizer(tokenizer=tokenize_zh)
```

### Construction du modèle

=> https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html

In [ ]:
def set_model_parameters(embedding_model):

    # Step 1 - extraction des embeddings
    embedding_model = embedding_model

    # Step 2 - réduction des dimensionalité
    umap_model = UMAP(angular_rp_forest=True, metric='cosine', n_components=10, n_neighbors=30, min_dist=0.1)

    # Step 3 - Cluster reduced embeddings
    hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=5, prediction_data=True, metric='euclidean', cluster_selection_method='eom')
    # hdbscan_model = KMeans(n_clusters=50)
    # hdbscan_model = sklearn.cluster.AgglomerativeClustering(n_clusters=50)

    # Step 4 - Tokenisation des topics

    stoplist = list(STOP_WORDS)
    ADDITIONAL_STOPWORDS = ["qu", "ya", "faut", "euh"]
    stoplist.extend(ADDITIONAL_STOPWORDS)

    vectorizer_model = CountVectorizer(stop_words=stoplist, ngram_range=(1, 3), max_df=0.5)

    # Step 5 - Création de la représentation des topics
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

    # Topic model
    return BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        language='french',
        n_gram_range=(1,3),
        nr_topics="auto"
        
    )

# Entrainement du modèle

On encode tous les documents de notre corpus.

In [ ]:
embedding_model = SentenceTransformer("dangvantuan/sentence-camembert-base")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

On crée une instance du modèle avec tous les paramètres définis et notre modèle d'embeddings.

In [ ]:
topic_model = set_model_parameters(embedding_model)

On ajuste le modèle aux documents de notre corpus et aux embeddings que nous avons préparés.

In [ ]:
topic_model.fit(docs, embeddings)

-1 est un "faux" cluster contenant toutes les valeurs aberrantes (*outliers*) ; il ne faut donc pas le prendre en compte. Un nom représentatif est attribué à chaque sujet. Le nom par défaut est un identifiant numérique suivi des mots-clés les plus représentatifs.

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(1)

In [ ]:
topic_model.generate_topic_labels(nr_words=20, topic_prefix=True, word_length=None, separator='  --  ')

In [ ]:
topic_model.save('last_bertopic.model')

# Exploration des prédictions du modèles

In [ ]:
topic_model = BERTopic.load('last_bertopic.model')

In [ ]:
# topics_to_merge = [[3,4,6],[3,15]]
# topic_model.merge_topics(docs, topics_to_merge)

barchart = topic_model.visualize_barchart(top_n_topics=17, title="Représentation des topics", width=400, n_words=7)
barchart.write_html('barchart.html')

In [ ]:
topic_labels_dict = {
    0:"L'opinion & le journalisme",
    1:"Financement & l'indépendance des médias",
    2:"Désinformation"
}

topic_model.set_topic_labels(topic_labels_dict)
barchart = topic_model.visualize_barchart(top_n_topics=17, custom_labels=True, title="Représentation des topics<", width=800, n_words=7)
barchart.write_html('barchart.html')

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
tree = topic_model.get_topic_tree(hierarchical_topics)
tree

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
hierarchy = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics, width=1200) # custom_labels=topic_labels_dict
hierarchy.write_html('hierarchy.html')

In [ ]:
heatmap = topic_model.visualize_heatmap(n_clusters=3, width=1200) # custom_labels=topic_labels_dict
heatmap.write_html('heatmap.html')

Visualisation du classement de tous les termes sur tous les sujets. Chaque sujet est représenté par un ensemble de mots. Cependant, ces mots ne représentent pas tous le sujet de manière égale. Cette visualisation montre combien de mots sont nécessaires pour représenter un sujet et à partir de quel seuil l'effet bénéfique de l'ajout de mots commence à diminuer.

Il ya deux axes :
- x : Représente le rang des termes dans chaque sujet. Le terme avec le score c-TF-IDF le plus élevé a un rang de 1, le deuxième terme a un rang de 2, et ainsi de suite.
- y : Montre le score c-TF-IDF des termes. Plus le score est élevé, plus le terme est représentatif du sujet.

In [ ]:
topic_model.visualize_term_rank(log_scale=True).write_html("term_rank.html")

In [ ]:
topic_model.visualize_topics().write_html("visualize_topics.html")

Enregistrer les résultats dans un csv

In [ ]:
PREDICTIONS_FILE = 'bertopic_topics_notebook.csv'

results = topic_model.get_document_info(docs=docs)
# results.to_csv()

# Latent Semantic Analysis (LSA)

LSA commence par créer une matrice terme-document. Les valeurs de la matrice sont généralement des scores TF-IDF qui pondèrent l'importance des mots dans chaque document. La matrice est ensuite décomposée en trois matrices à l'aide de la SVD. Ces matrices capturent les relations entre les termes et les documents. Enfin, en sélectionnant uniquement les premières composantes singulières, LSA réduit la dimensionalité tout en conservant les informations les plus importantes, ce qui permet de découvrir les sujets sous-jacents.

```python
embedding_model = sklearn.pipeline.make_pipeline(
    sklearn.feature_extraction.text.CountVectorizer(analyzer="word", tokenizer=lambda x: x.split(" ")),
    sklearn.decomposition.TruncatedSVD(10)
)
```

=> https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD

In [ ]:
def set_model_parameters_lsa():

    # Step 1 - extraction des embeddings
    embedding_model = sklearn.pipeline.make_pipeline(
        sklearn.feature_extraction.text.CountVectorizer(analyzer="word", tokenizer=lambda x: x.split(" ")),
        sklearn.decomposition.TruncatedSVD(n_components=15)
    )

    # Step 2 - réduction des dimensionalité
    umap_model = UMAP(angular_rp_forest=True, metric='cosine', n_components=10, n_neighbors=20, min_dist=0.1)

    # Step 3 - Cluster reduced embeddings
    hdbscan_model = HDBSCAN(min_cluster_size=10, min_samples=5, prediction_data=True, metric='euclidean', cluster_selection_method='eom')
    # hdbscan_model = KMeans(n_clusters=50)
    # hdbscan_model = sklearn.cluster.AgglomerativeClustering(n_clusters=50)

    # Step 4 - Tokenisation des topics
    stoplist = list(STOP_WORDS)
    ADDITIONAL_STOPWORDS = ["qu", "ya", "faut", "euh"]
    stoplist.extend(ADDITIONAL_STOPWORDS)

    vectorizer_model = CountVectorizer(stop_words=stoplist, ngram_range=(1, 2))

    # Step 5 - Création de la représentation des topics
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

    # Topic model
    return BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        language='french',
        n_gram_range=(1,2),
    )

In [ ]:
topic_model = set_model_parameters_lsa()

In [ ]:
topic_model.fit(docs)

In [ ]:
topic_model.generate_topic_labels(nr_words=20, topic_prefix=True, word_length=None, separator='  --  ')

In [ ]:
topic_model.get_topic_info()